In [1]:
import requests
import re 
from bs4 import BeautifulSoup 
from bs4 import UnicodeDammit

evergreen_url = 'https://www.trustpilot.com/review/egcu.org'

response = requests.get(evergreen_url)

soup = BeautifulSoup(response.content, "html.parser")

In [2]:
#Details

import pandas as pd

company_details = soup.find_all('div')

data = []

company = company_details[0] 

name_element = company.find('span', attrs={'class': 'typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM'})
reviews_element = company.find('p', attrs={'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17'})
star_elements = company.find_all('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_cell__qnPHy styles_percentageCell__cHAnb'})

if name_element and reviews_element:
    name = name_element.text.strip()
    reviews = re.search(r'\d+', reviews_element.text.strip()).group()
    stars = [star_element.text.strip() for star_element in star_elements]

    data.append([name, reviews] + stars)

columns = ['name', 'reviews', 'star_5', 'star_4', 'star_3', 'star_2', 'star_1']

df_company_details = pd.DataFrame(data, columns=columns)

# Export DataFrame to CSV
df_company_details.to_csv('company_details.csv', index=False)

print(df_company_details)

                     name reviews star_5 star_4 star_3 star_2 star_1
0  Evergreen Credit Union     222    97%     2%    <1%    <1%    <1%


In [3]:
#Reviews

import pandas as pd

reviews = soup.find_all('div', attrs={'class': 'styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ'})

# Initialize lists to store the data
data = []

# Extract the data from each review container
for review in reviews:
    # Extract review stars
    review_stars = review.find_all('div', attrs={'class': 'star-rating_starRating__4rrcf star-rating_medium__iN6Ty'})
    stars = [stars.find('img')['alt'].replace('Rated ', '').replace(' stars', '') for stars in review_stars]

    # Extract other review details
    review_dates = review.find('time', attrs={'class': '', 'data-service-review-date-time-ago': 'true'})
    review_title = review.find('h2', attrs={'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17'})
    reviewer_name = review.find('span', attrs={'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'})
    review_text = review.find('p', attrs={'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
    experience_date = review.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17'})
    review_reply_text = review.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX'})
    reply_date_ = review.find('time', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_'})

    # Extract the values or set them as None if not found
    star = stars[0] if stars else None
    title = review_title.text.strip() if review_title else None
    reviewer = reviewer_name.text.strip() if reviewer_name else None
    text = review_text.text.strip() if review_text else None
    experience = experience_date.text.split(':')[-1].strip() if experience_date else None
    review_date = review_dates.get('datetime').split('T')[0].strip() if review_dates else None
    reply_date = reply_date_.text.strip() if reply_date_ else None
    reply_text = review_reply_text.text.strip() if review_reply_text else None

    data.append([star, title, reviewer, text, experience, review_date, reply_date, reply_text])

# Define column names for the DataFrame
columns = ['review_star', 'review_title', 'reviewer_name', 'review_text', 'experience_date', 'review_date', 'reply_date', 'review_reply_text']

# Create a DataFrame from the data
df_reviews = pd.DataFrame(data, columns=columns)

# Export DataFrame to CSV
df_reviews.to_csv('reviews.csv', index=False)

df_reviews.head(20)

,review_star,review_title,reviewer_name,review_text,experience_date,review_date,reply_date,review_reply_text
0,5 out of 5,ATV Loan Experience,Chris L.,I worked with Isaiah on a ATV loan. The entire...,"July 18, 2023",2023-07-19,Updated 2 days ago,"Hi Chris,Thank you for sharing your Evergreen ..."
1,5 out of 5,Excellent Experience Purchasing a Car,Christine,I worked with Kacie and was referred over by R...,"July 18, 2023",2023-07-18,2 days ago,"Hi Christine,Thank you for leaving a review! W..."
2,5 out of 5,Old Guy Gets Auto Loan. Who Knew??,Paul Bellefeuille,I had not applied for an auto loan in over 25 ...,"July 18, 2023",2023-07-19,2 days ago,"Hi Paul,Congratulations! It's never too late t..."
3,5 out of 5,Top Notch Staff,Kate,We have had nothing but top notch service with...,"June 30, 2022",2023-06-30,"Jun 30, 2023","Hi Kate,Thanks for leaving your positive revie..."
4,5 out of 5,Fast and efficient loan process,customer,"Fast and efficient loan process, thank you Isa...","June 30, 2023",2023-07-17,3 days ago,"Hi there,We're glad Isaiah was able to help yo..."
5,5 out of 5,This is the best evergreen credit union,Hanii Rajput,None,"July 17, 2023",2023-07-20,19 hours ago,"Hi Hanii,Thank you! We appreciate the complime..."
6,5 out of 5,Easy process and friendlier people,patrick,From my initial inquiry to securing a membersh...,"June 27, 2023",2023-06-30,"Jul 3, 2023","Hi Patrick,Thank you for sharing your experien..."
7,5 out of 5,I was very impressed with the car loan…,Lianie McGowan,I was very impressed with the car loan process...,"July 05, 2023",2023-07-07,3 days ago,"Hi Lianie,Thank you for sharing your Evergreen..."
8,5 out of 5,She was wonderful a true joy to speak…,MD,She was wonderful a true joy to speak with and...,"June 29, 2023",2023-06-29,"Jun 30, 2023","Hi there,Thank you for your feedback! If you h..."
9,5 out of 5,EGCU Above and Beyond in Customer Service,Lisa Dominicus,Today I had one of the best customer service e...,"June 13, 2023",2023-06-14,"Jun 15, 2023","Hi Lisa,This is such a great experience! Thank..."
